In [1]:
import pandas as pd
import numpy as np
import csv
from utility import *
import seaborn as sb
import operator

In [2]:
df_train_query=pd.read_csv('dataset-train/split_train_queries_with_category_labels.csv',sep=";")

In [3]:
#df_train_query['searchstring.tokens']=df_train_query['searchstring.tokens'].fillna('#')

In [4]:
category=pd.read_csv('dataset-train/product-categories.csv',sep=';')

In [5]:
print category.columns
print "#unique categories:",len(category['categoryId'].unique())

Index([u'itemId', u'categoryId'], dtype='object')
#unique categories: 1217


In [6]:
print "query-less sessions:",sum(df_train_query['searchstring.tokens'].isnull())
print len(df_train_query['searchstring.tokens'])

query-less sessions: 0
636160


In [7]:
#df_train_query[df_train_query['searchstring.tokens'].isnull()]

In [8]:
df_train_query_less=df_train_query[df_train_query['searchstring.tokens']=='#']
df_train_query_full=df_train_query[df_train_query['searchstring.tokens']!='#']

In [9]:
#tmp=df_train_query_less[df_train_query_less['categoryId']==0]
#tmp.head(20)
#tmp.head(10)

In [10]:
#check to see whether a query was targeting towards a product category
#l=list(str(df_train_query[df_train_query['queryId']==1]['items'][0]).split(','))
"""dft=df_train_query[df_train_query['queryId']==94]['items']
for p in dft.all().split(','):
    print category[category['itemId']==int(p)]#.categryId#['categoryId']"""

"dft=df_train_query[df_train_query['queryId']==94]['items']\nfor p in dft.all().split(','):\n    print category[category['itemId']==int(p)]#.categryId#['categoryId']"

In [11]:
df_test_query=pd.read_csv('dataset-train/split_test_queries_with_category_labels.csv',sep=';')

In [12]:
#df_test_query['searchstring.tokens']=df_test_query['searchstring.tokens'].fillna('#')

In [13]:
#add new column to split_train_queries.csv with category target variable
"""target_category=[]
total_rows=len(df_test_query)
itr=1
for i,row in df_test_query.iterrows():
    print "\rrow_num:"+str(itr)+" "+str(total_rows),
    if (row['searchstring.tokens']!='#') or (row['searchstring.tokens']=='#' and row['categoryId']==0):
        items=row['items'].split(',')
        category_count={}
        for item in items:
            cat=category[category['itemId']==int(item)]['categoryId'].values[0]
            if cat not in category_count:
                category_count[cat]=0
            category_count[cat]+=1
        #print category_count
        l=sorted(category_count.items(), key=operator.itemgetter(1),reverse=True)
        target_category.append(l[0][0])
    else:
        #print("here")
        target_category.append(row['categoryId'])
    itr+=1"""
        

'target_category=[]\ntotal_rows=len(df_test_query)\nitr=1\nfor i,row in df_test_query.iterrows():\n    print "\rrow_num:"+str(itr)+" "+str(total_rows),\n    if (row[\'searchstring.tokens\']!=\'#\') or (row[\'searchstring.tokens\']==\'#\' and row[\'categoryId\']==0):\n        items=row[\'items\'].split(\',\')\n        category_count={}\n        for item in items:\n            cat=category[category[\'itemId\']==int(item)][\'categoryId\'].values[0]\n            if cat not in category_count:\n                category_count[cat]=0\n            category_count[cat]+=1\n        #print category_count\n        l=sorted(category_count.items(), key=operator.itemgetter(1),reverse=True)\n        target_category.append(l[0][0])\n    else:\n        #print("here")\n        target_category.append(row[\'categoryId\'])\n    itr+=1'

In [14]:
#target_category

In [15]:
#df_train_query.head(20)

In [16]:
#df_test_query['target_category']=target_category

In [17]:
#df_test_query.to_csv('dataset-train/split_test_queries_with_category_labels.csv',sep=';',index=False)

In [18]:
#df_test_query[df_test_query['categoryId']!=0].head(10)

In [19]:
df_test_query_less=df_test_query[df_test_query['searchstring.tokens']=='#']
df_test_query_full=df_test_query[df_test_query['searchstring.tokens']!='#']

In [20]:
df_query_full=df_test_query_full.append(df_train_query_full)

In [21]:
#print len(df_query_full['queryId'].unique())
#print len(df_query_full)
print len(df_train_query[df_train_query['searchstring.tokens']!='#'])+len(df_test_query[df_test_query['searchstring.tokens']!='#'])

51888


In [22]:
#df_query_full.to_csv('dataset-train/split_query_full_with_category_labels.csv',sep=';',index=False)
df_query_full=pd.read_csv('dataset-train/split_query_full_with_category_labels.csv',sep=';')

In [23]:
from sklearn.cross_validation import train_test_split

In [24]:
print len(df_query_full)
df_query_full_train, df_query_full_test = train_test_split(df_query_full, test_size = 0.2)
print len(df_query_full_train)
print len(df_query_full_test)

51888
41510
10378


In [25]:
from sklearn.feature_extraction.text import CountVectorizer


In [26]:
vectorizer=CountVectorizer()

In [27]:
vectorizer.fit(df_query_full['searchstring.tokens'])

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [28]:
#df_query_full.head()

In [29]:
#print vectorizer.transform(pd.Series("16655,16655"))
#print vectorizer.transform(df_query_full[df_query_full['queryId']==3]['searchstring.tokens'])

In [30]:
bag_of_words_train=vectorizer.transform(df_query_full_train['searchstring.tokens'])
bag_of_words_test=vectorizer.transform(df_query_full_test['searchstring.tokens'])

In [47]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model.perceptron import Perceptron

In [55]:
#model=MultinomialNB() #70+
#model=XGBClassifier(learning_rate =0.1,n_estimators=100,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=27)
#model=LogisticRegression(multi_class='ovr') # 83+
#model=RandomForestClassifier(n_estimators=50)
#model=Perceptron(n_iter=100)#92+
model=SGDClassifier(n_iter=100,loss='modified_huber')#89+ best test performance

In [56]:
model.fit(bag_of_words_train,df_query_full_train['target_category'])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=100,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, verbose=0, warm_start=False)

In [57]:
labels_train=model.predict(bag_of_words_train)
labels_test=model.predict(bag_of_words_test)

In [58]:
#training accuracy
print sum(df_query_full_train['target_category']==labels_train)/float(len(labels_train))
print sum(df_query_full_test['target_category']==labels_test)/float(len(labels_test))

0.928667790894
0.806417421468


In [63]:

model.predict_proba(bag_of_words_test[0]).shape

(1, 524)

In [ ]:
# Loading item views; taking itemId column
item_views = pd.read_csv('dataset-train/train-item-views.csv', sep=';')
print('Item views', len(item_views))

# Loading clicks; taking itemId column
clicks = pd.read_csv('dataset-train/train-clicks_with_usrId.csv', sep=';')
print('Clicks', len(clicks))

# Loading purchases; taking itemId column
purchases = pd.read_csv('dataset-train/train-purchases.csv', sep=';')
print('Purchases', len(purchases))

In [ ]:
#l=clicks.merge(df_train_query[['queryId','userId']], on='queryId', how='inner')['userId']

In [ ]:
#clicks['userId']=l

In [ ]:
#clicks.head()

In [ ]:
#df_train_query[df_train_query['queryId']==1]['items'][0]

In [ ]:
#clicks.to_csv('dataset-train/train-clicks_with_usrId.csv',sep=';',index=False)

In [ ]:
#clicks_=pd.read_csv('dataset-train/train-clicks_with_usrId.csv',sep=';')

In [ ]:
users=df_train_query['userId'].unique()

In [ ]:
print item_views.columns
print clicks.columns
print purchases.columns
print df_train_query.columns

In [ ]:
len(users)

In [ ]:
user_product_pref={}
i=1
#for user in users:
for user in [6.0,10.0]:
    print "\ruser "+str(i)+":"+str(user),
    prod_pref={}
    #df_temp=df_train_query[df_train_query['userId']==user]
    user_item_views=item_views[item_views['userId']==user]['itemId']
    user_purchases=purchases[purchases['userId']==user]['itemId']
    user_clicks=clicks[clicks['userId']==user]['itemId']
    #for ii,row in df_temp.iterrows():        
    for cost, container in enumerate([user_item_views, user_clicks, user_purchases]):
        for prod in container.values:
            product = str(prod)
            print(user,product,cost)
            if product not in prod_pref:
                prod_pref[product] = cost
            else:
                prod_pref[product] += cost
    user_product_pref[user]=prod_pref
    i+=1

In [ ]:
#save_object(user_product_cost,'user_product_cost.pkl')
user_product_cost=load_object('user_product_cost.pkl')

In [ ]:
#df_train_query.head(100)
#print user_product_cost[6.0]
print user_product_cost[2110.0]


In [ ]:
#item_views[item_views['userId']==10.0]
item_views[item_views['userId']==6.0]

In [ ]:
clicks[clicks['userId']==6.0]

In [ ]:
#df_views=pd.read_csv('dataset/train-item-views.csv',sep=';')

In [ ]:
#df_views.loc[df_views['sessionId']==16]
#df_train.loc[df_train['sessionId']==2]

In [ ]:
#df_views.loc[df_views['sessionId']==2]

In [ ]:
#df_test=pd.read_csv('dataset/split_test_queries.csv',sep=';')

In [ ]:
#df_test.head()